In [2]:
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_absolute_error
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import random
from pathlib import Path
import os
from datetime import datetime
import pickle

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [31]:
device

device(type='cuda')

## Load data and prepare for inference / eval

In [26]:
data_file = os.path.join(os.getcwd(), "CAISO_Data_2019_2021_NN.with_historical_projections.csv")
# read in the processed data
data_df = pd.read_csv(data_file, index_col=0)
data_df.index = pd.to_datetime(data_df.index)
# add temporal features
## Temporal features
data_df.loc[:,"Day_of_Year"] = [instant.timetuple().tm_yday for instant in data_df.index]
data_df.loc[:,"Hour"] = data_df.index.hour

# load the scaler to use for scaling the data prior to inference
scaler_file = os.path.join(os.getcwd(), "FF_models", "scaler.pkl")
with open(scaler_file, "rb") as f:
    scaler = pickle.load(f)

Not all hours have a projected value for day-ahead or hour-ahead demand/vre. Only evaluate on the hours that have projected values for both demand and vre.

In [30]:
data_sets = {"hour_ahead": {"demand_col": "Hour_Ahead_Forecast_Demand",
                            "vre_col": "Hour_Ahead_Forecast_VRE",
                           }, 
             "day_ahead": {"demand_col": "Day_Ahead_Forecast_Demand",
                           "vre_col": "Day_Ahead_Forecast_VRE",
                          }
            }
for name, data_set_info in data_sets.items():
    demand_col = data_set_info["demand_col"]
    vre_col = data_set_info["vre_col"]
    # get rows that have a projection for both demand and vre
    mask = (~pd.isna(data_df.loc[:,demand_col])) & (~pd.isna(data_df.loc[:,demand_col]))
    data_set_info["data_set_mask"] = mask
    
    # load this data to tensors for input features and labels
    feature_cols = [demand_col, vre_col, 'Hour', 'Day_of_Year']
    label_col = 'delta_Total_CO2_Emissions'
    X = torch.tensor(data_df.loc[mask, feature_cols].values.astype(np.float32))
    y = torch.tensor(data_df.loc[mask, label_col].values.astype(np.float32)).to(device)
    # scale input features
    X = torch.tensor(scaler.transform(X)).to(device)
    data_set_info["X"] = X
    data_set_info["y"] = y
    
    # also save the d_VRE and d_demand (TODO, go compute those in the pre-processing step since we need the additional info of the hour before our earliest hour)

In [32]:
print("Number of example hours for which we have both projected demand and VRE values...")
for name, data_info in data_sets.items():
    print(f"{name}: {len(data_info['y'])}")

Number of example hours for which we have both projected demand and VRE values...
hour_ahead: 26303
day_ahead: 26302


## Load saved model

In [36]:
def get_model(n_input, hidden_dims, n_out, dropout_p):
    
    layers = [nn.Linear(n_input, hidden_dims[0]),
              nn.BatchNorm1d(hidden_dims[0]),
              nn.ReLU(),
              nn.Dropout(dropout_p)
             ]
    for layer in range(len(hidden_dims)-1):
        cur_hidden, next_hidden = hidden_dims[layer], hidden_dims[layer+1]
        layers.extend([nn.Linear(cur_hidden, next_hidden),
                       nn.BatchNorm1d(next_hidden),
                       nn.ReLU(),
                       nn.Dropout(dropout_p)
                      ])
    layers.append(nn.Linear(hidden_dims[-1], n_out))
    
    model = nn.Sequential(*layers)
    return model


In [35]:
best_model_file = os.path.join(os.getcwd(), "ff_models", \
                               "base4_features", "2023_01_27-04_25_05_PM", \
                               "epoch=30059,r2=0.8836,Invalids=0.pth")

In [38]:
hidden_dims = [512,256]
n_out = 3 # intercept term
dropout_p = 0.5
n_input = len(feature_cols)

model = get_model(n_input, hidden_dims, n_out, dropout_p)
model.to(device)
model.load_state_dict(torch.load(best_model_file))
model.eval()

Sequential(
  (0): Linear(in_features=4, out_features=512, bias=True)
  (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Dropout(p=0.5, inplace=False)
  (4): Linear(in_features=512, out_features=256, bias=True)
  (5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): Dropout(p=0.5, inplace=False)
  (8): Linear(in_features=256, out_features=3, bias=True)
)

In [ ]:
for name, data_set_info in data_sets.items():
    X, y = data_set_info["X"], data_set_info["y"]
    # inference
    pred_coeff = model(X.float()).cpu()
    # split into mefs mdfs and intercept terms 
    mefs = pred_coeff[:, 0]
    mdfs = pred_coeff[:, 1]
    intercepts = pred_coeff[:, 2]
    data_set_info["pred_mef"] = mefs
    data_set_info["pred_mdf"] = mdfs
    data_set_info["pred_intercepts"] = intercepts
    

In [8]:
def get_r_squared(pred_coeff, CAISO_Data, bias_term):
    coeff_df=pd.DataFrame(data=pred_coeff.detach().numpy(), columns=['MEF', 'MDF', 'Intercept'])
    pred_delta_total_co2_emissions = coeff_df['MEF'].values * CAISO_Data['delta_Load'] \
                                   + coeff_df['MDF'].values * CAISO_Data['delta_VRE']
    if bias_term:
        pred_delta_total_co2_emissions += coeff_df['Intercept'].values
    r2 = r2_score(CAISO_Data['delta_Total_CO2_Emissions'], pred_delta_total_co2_emissions)
    return r2

def get_mean_abs_err(pred_coeff, CAISO_Data, bias_term):
    coeff_df=pd.DataFrame(data=pred_coeff.detach().numpy(), columns=['MEF', 'MDF', 'Intercept'])
    pred_delta_total_co2_emissions = coeff_df['MEF'].values * CAISO_Data['delta_Load'] \
                                   + coeff_df['MDF'].values * CAISO_Data['delta_VRE']
    if bias_term:
        pred_delta_total_co2_emissions += coeff_df['Intercept'].values
    mean_abs_err = mean_absolute_error(CAISO_Data['delta_Total_CO2_Emissions'], pred_delta_total_co2_emissions)
    return mean_abs_err

def get_count_invalid_preds(pred_coeff):
    # preds=pred_coeff.detach().numpy()
    count_neg_MEFs = torch.sum(pred_coeff[:,0] <= 0).item() #sum(preds[:,0] <= 0) # MEF must be greater than 0
    count_pos_MDFs = torch.sum(pred_coeff[:,1] > 0).item() #sum(preds[:,1] > 0)  # MDF must be less than or equal to 0
    return count_neg_MEFs, count_pos_MDFs

## Inference on all data

In [19]:


results_str = "R Squared:"
results_str += f"\n\tTrain: {get_r_squared(train_pred_coeff, CAISO_train, bias_term):.4f}"
results_str += f"\n\tVal: {get_r_squared(val_pred_coeff, CAISO_val, bias_term):.4f}"
results_str += f"\n\tTest: {get_r_squared(test_pred_coeff, CAISO_test, bias_term):.4f}"
results_str += "\nMean Absolute Error:"
results_str += f"\n\tTrain: {get_mean_abs_err(train_pred_coeff, CAISO_train, bias_term):.2f}"
results_str += f"\n\tVal: {get_mean_abs_err(val_pred_coeff, CAISO_val, bias_term):.2f}"
results_str += f"\n\tTest: {get_mean_abs_err(test_pred_coeff, CAISO_test, bias_term):.2f}"
results_str += "\nCount Invalid Values Predicted:"
invalid_train_MEFs, invalid_train_MDFs = get_count_invalid_preds(train_pred_coeff)
invalid_val_MEFs, invalid_val_MDFs = get_count_invalid_preds(val_pred_coeff)
invalid_test_MEFs, invalid_test_MDFs = get_count_invalid_preds(test_pred_coeff)
results_str += f"\n\tTrain: Invalid MEFs={invalid_train_MEFs}, Invalid MDFs={invalid_train_MDFs}"
results_str += f"\n\tVal: Invalid MEFs={invalid_val_MEFs}, Invalid MDFs={invalid_val_MDFs}"
results_str += f"\n\tTest: Invalid MEFs={invalid_test_MEFs}, Invalid MDFs={invalid_test_MDFs}"

with open(os.path.join(best_model_dir, "eval_results.txt"), 'w+') as f:
    f.write(results_str)
print(results_str)

R Squared:
	Train: 0.8951
	Val: 0.8836
	Test: 0.8845
Mean Absolute Error:
	Train: 129444.94
	Val: 136886.79
	Test: 132747.95
Count Invalid Values Predicted:
	Train: Invalid MEFs=0, Invalid MDFs=0
	Val: Invalid MEFs=0, Invalid MDFs=0
	Test: Invalid MEFs=0, Invalid MDFs=0


### Put the MEFs and MDFs from all sets back together and in order into the original DF for viewing

In [20]:
all_preds_w_timestamps = list(zip(CAISO_val.index, val_pred_coeff.detach().numpy())) \
                        + list(zip(CAISO_train.index, train_pred_coeff.detach().numpy())) \
                        + list(zip(CAISO_test.index, test_pred_coeff.detach().numpy()))
all_preds_w_timestamps.sort(key=lambda pair: pair[0])
all_preds_ordered = np.array([pair[1] for pair in all_preds_w_timestamps])

In [21]:
all_MEFs_ordered = all_preds_ordered[:,0]
all_MDFs_ordered = all_preds_ordered[:,1]
all_intercepts_ordered = all_preds_ordered[:,2]

In [22]:
CAISO_Data.loc[:,"MEF"] = all_MEFs_ordered
CAISO_Data.loc[:,"MDF"] = all_MDFs_ordered
if bias_term:
    CAISO_Data.loc[:,"Intercept"] = all_intercepts_ordered

#calculate some error stuff. rn i am thinking R2 is the best measure of error
d_emissions = CAISO_Data.loc[:,'MEF'] * CAISO_Data.loc[:,'delta_Load'] \
            + CAISO_Data.loc[:,'MDF'] * CAISO_Data.loc[:,'delta_VRE']
if bias_term:
    d_emissions += CAISO_Data.loc[:,"Intercept"]
CAISO_Data.loc[:,'Predicted_delta_Total_CO2_Emissions'] = d_emissions
CAISO_Data.loc[:,'Error']=CAISO_Data.loc[:,'Predicted_delta_Total_CO2_Emissions']-CAISO_Data.loc[:,'delta_Total_CO2_Emissions']
CAISO_Data.loc[:,'%_Error']=np.abs(CAISO_Data.loc[:,'Error'])/np.abs(CAISO_Data.loc[:,'delta_Total_CO2_Emissions'])
print("Whole Data Set:")
print(f"\tMean Emissions Change = {np.mean(np.abs(CAISO_Data['delta_Total_CO2_Emissions'])):.2f}")
print(f"\tR Squared = {r2_score(CAISO_Data['delta_Total_CO2_Emissions'], CAISO_Data['Predicted_delta_Total_CO2_Emissions']):.4f}")
print(f"\tMean Absolute Error = {mean_absolute_error(CAISO_Data['delta_Total_CO2_Emissions'], CAISO_Data['Predicted_delta_Total_CO2_Emissions']):.2f}")

Whole Data Set:
	Mean Emissions Change = 413162.83
	R Squared = 0.8907
	Mean Absolute Error = 131593.91


In [23]:
CAISO_Data.head()

,Load,Net Load,Total_CO2_Emissions,Total_SO2_Emissions,Total_NOX_Emissions,VRE,delta_Load,delta_Net_Load,delta_Total_CO2_Emissions,delta_Total_SO2_Emissions,delta_Total_NOX_Emissions,delta_VRE,Day_of_Year,Hour,MEF,MDF,Intercept,Predicted_delta_Total_CO2_Emissions,Error,%_Error
2019-01-01 00:00:00,22822.964472,20502.358502,5.103942e+06,425.327933,1632.821698,2320.593616,-1285.054865,-1255.110267,-337029.794143,-24.142180,-98.530417,-29.956338,1,0,397.956238,-403.021606,2859.093994,-496463.453856,-159433.659713,0.473055
2019-01-01 01:00:00,21879.620618,19606.836908,4.867578e+06,404.315852,1557.650531,2272.780097,-944.689268,-896.922625,-243021.833700,-21.594332,-76.882199,-47.757034,1,1,350.681824,-340.116089,2318.183838,-312724.235908,-69702.402208,0.286815
2019-01-01 02:00:00,21257.454020,19056.267637,4.723101e+06,383.695714,1496.197481,2201.182455,-614.641020,-545.206677,-144846.797503,-20.952957,-61.425638,-69.432802,1,2,316.226929,-288.244049,1798.801270,-172553.648694,-27706.851191,0.191284
2019-01-01 03:00:00,20974.800758,18871.418601,4.693112e+06,380.561848,1466.329836,2103.388502,-281.391674,-191.565227,-24776.569759,-2.164379,-28.118595,-89.815150,1,3,311.336304,-266.898071,1289.010376,-62346.943092,-37570.373333,1.516367
2019-01-01 04:00:00,20327.083333,18012.666667,5.032423e+06,711.911968,2391.657870,2314.666667,30.416667,74.416667,49254.136541,69.703951,71.970840,-43.916667,1,4,315.807770,-248.001999,944.050781,21441.291564,-27812.844977,0.564680


In [24]:
CAISO_Data.to_csv(os.path.join(best_model_dir, "CAISO_Data_2019_2021_NN.with_coeff_preds.csv"))

In [25]:
len([val for val in CAISO_Data.loc[:,"MEF"] if val <=0])

0

In [26]:
len([val for val in CAISO_Data.loc[:,"MEF"] if val >600])

2

In [27]:
CAISO_Data.loc[:,"MEF"].max()

631.6832885742188